<a href="https://colab.research.google.com/github/Pandaklez/masters-hse-proga/blob/master/hw2Klezovich_ngrams.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Задание 1.

В тетрадке реализована биграмная языковая модель (при генерации учитывается информация только о 1 предыдущем слове). Реализуйте триграмную модель и сгенерируйте несколько текстов. Сравните их с текстами, сгенерированными биграмной моделью. 
Можно использовать те же тексты, что в семинаре, или взять какой-то другой (на английском или русском языке).  

Делать это задание будет легче после прочтения первых 7 страниц вот этой главы из Журафского - https://web.stanford.edu/~jurafsky/slp3/3.pdf

Я частично копирую текст семинарской тетрадки, прошу прощения за путаницу.

Возьмем два разных корпуса: новостной и сообщения с 2ch.

In [1]:
# !!! двач не самое приятное место, большое количество текстов в этом корпусе токсичные
dvach = open('2ch_corpus.txt').read()
# !!! двач не самое приятное место, большое количество текстов в этом корпусе токсичные

news = open('lenta.txt').read()

Напишем простую функцию для нормализации. 

In [2]:
! pip install razdel

In [2]:
from string import punctuation
from razdel import sentenize
from razdel import tokenize as razdel_tokenize
import numpy as np

def normalize(text):
    normalized_text = [word.text.strip(punctuation) for word \
                                                            in razdel_tokenize(text)]
    normalized_text = [word.lower() for word in normalized_text if word and len(word) < 20 ]
    return normalized_text


Сравним тексты по токенам

In [3]:
norm_dvach = normalize(dvach)
norm_news = normalize(news)

In [8]:
print("Длина корпуса токсичных постов в токенах -", len(norm_dvach))
print("Длина корпуса новостных текстов в токенах - ", len(norm_news))

Длина корпуса токсичных постов в токенах - 1858941
Длина корпуса новостных текстов в токенах -  1505789


И по уникальным токенам

In [5]:
print("Уникальных токенов в корпусе токсичных постов -", len(set(norm_dvach)))
print("Уникальный токенов в корпусе новостных текстов - ", len(set(norm_news)))

Уникальных токенов в корпусе токсичных постов - 189515
Уникальный токенов в корпусе новостных текстов -  116302


Посчитаем, сколько раз встречаются слова и выведем самые частотные.

In [3]:
from collections import Counter

In [7]:
vocab_dvach = Counter(norm_dvach)
vocab_news = Counter(norm_news)


Сравнивать употребимость конкретных слов в разных текстах в абсолютных числах неудобно. Нормализуем счётчики на размеры текстов. Так у нас получается вероятность слова.

In [8]:
probas_dvach = Counter({word:c/len(norm_dvach) for word, c in vocab_dvach.items()})
probas_dvach.most_common(5)

[('и', 0.030066580918921042),
 ('в', 0.02628001641794979),
 ('не', 0.02506911192985684),
 ('на', 0.015955320798239428),
 ('что', 0.014345802260534357)]

In [9]:
probas_news = Counter({word:c/len(norm_news) for word, c in vocab_news.items()})
probas_news.most_common(5)

[('в', 0.04808907489694771),
 ('и', 0.0221080111489724),
 ('на', 0.018883123731146926),
 ('по', 0.012943380513471676),
 ('что', 0.011310349590812525)]

Эти вероятности уже можно использовать, чтобы ответить на вопрос - это предложение больше подходит для новостей или для анонимного форума?

In [10]:
phrase = 'Технические возможности устаревшего российского судна не позволили разгрузить его у терминала'

prob = Counter({'news':0, 'dvach':0})

for word in normalize(phrase):
    prob['dvach'] += probas_dvach.get(word, 0)
    prob['news'] += probas_news.get(word, 0)



In [11]:
prob.most_common()

[('dvach', 0.034328147047162874), ('news', 0.014186582582287426)]

In [12]:
phrase = 'как вы смотрите эту залупу, серьезно. в чем прикол ваще это смотреть?'

prob = Counter({'news':0, 'dvach':0})

for word in normalize(phrase):
    prob['dvach'] += probas_dvach.get(word, 0)
    prob['news'] += probas_news.get(word, 0)



In [13]:
prob.most_common()

[('news', 0.05619313197267346), ('dvach', 0.04750823183737408)]

Результаты получаются не очень точные. Возможно это из-за того, что мы считаем слова независимыми друг от друга. А это очевидно не так

По-хорошему вероятность последовательности нужно расчитывать по формуле полной вероятности. Но у нас не очень большие тексты и мы не можем получить вероятности для длинных фраз (их просто может не быть в текстах). Поэтому мы воспользуемся предположением Маркова и будем учитывать только предыдущее слово.

Чтобы расчитать вероятность с таким предположением, нам достаточно найти количество вхождений для каждого биграмма.

In [4]:
from nltk.tokenize import sent_tokenize

# вот тут начинается интересное. 
def ngrammer(tokens, n=2):
    ngrams = []
    for i in range(0,len(tokens)-n+1):
        ngrams.append(' '.join(tokens[i:i+n]))
    return ngrams

Для того, чтобы у нас получились честные вероятности и можно было посчитать вероятность первого слова, нам нужно добавить тэг маркирующий начало предложений \< start \>

Дальше мы попробуем сгенерировать текст, используя эти вероятности, и нам нужно будет когда-то остановится. Для этого добавим тэг окончания \< end \>

Ну и поделим все на предложения

In [22]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [5]:
len(dvach)

11638405

In [14]:
sentences_dvach = [['<start>'] + normalize(text) + ['<end>'] for text in sent_tokenize(dvach[:100000])]
sentences_news = [['<start>'] + normalize(text) + ['<end>'] for text in sent_tokenize(news[:100000])]

In [15]:
len(sentences_dvach)

1450

Вот на этом этапе к составлению биграммов я добавляю составление триграммов. Чтобы потом можно было сравнить два типа этих моделей.

Так же я беру только часть предложений, потому что иначе у меня умирает память на этапе генерации текстов.

In [16]:
unigrams_dvach = Counter()
bigrams_dvach = Counter()
trigrams_dvach = Counter()

for sentence in sentences_dvach[:10000]:
    unigrams_dvach.update(sentence)
    bigrams_dvach.update(ngrammer(sentence))
    trigrams_dvach.update(ngrammer(sentence, n=3))


unigrams_news = Counter()
bigrams_news = Counter()
trigrams_news = Counter()

for sentence in sentences_news[:10000]:
    unigrams_news.update(sentence)
    bigrams_news.update(ngrammer(sentence))
    trigrams_news.update(ngrammer(sentence, n=3))


In [17]:
len(unigrams_dvach)

5450

In [10]:
bigrams_news.most_common(10)

[('<start> в', 959),
 ('<start> по', 791),
 ('<start> как', 577),
 ('риа новости', 361),
 ('как сообщает', 284),
 ('по словам', 247),
 ('<start> на', 233),
 ('об этом', 226),
 ('<start> об', 208),
 ('<start> однако', 205)]

In [28]:
trigrams_news.most_common(10)

[('<start> об этом', 1579),
 ('<start> по словам', 1549),
 ('сообщает риа новости', 1324),
 ('со ссылкой на', 1243),
 ('риа новости <end>', 1228),
 ('<start> кроме того', 1070),
 ('<start> как сообщает', 1064),
 ('<start> напомним что', 1006),
 ('по его словам', 899),
 ('<start> по его', 868)]

Чтобы посчитать условную вероятность мы можем поделить количество вхождений на количество вхождений первого слова.

In [19]:
phrase = 'Технические возможности устаревшего российского судна не позволили разгрузить его у терминала'
# phrase = 'как вы смотрите эту залупу, серьезно. в чем прикол ваще это смотреть?'
prob = Counter()
for ngram in ngrammer(['<start>'] + normalize(phrase) + ['<end>']):
    word1, word2 = ngram.split()
    
    if word1 in unigrams_dvach and ngram in bigrams_dvach:
        prob['dvach'] += np.log(bigrams_dvach[ngram]/unigrams_dvach[word1])
    else:
        prob['dvach'] += np.log(0.001)
    
    if word1 in unigrams_news and ngram in bigrams_news:
        prob['news'] += np.log(bigrams_news[ngram]/unigrams_news[word1])
    else:
        prob['news'] += np.log(0.001)

prob['news'] = np.exp(prob['news'])
prob['dvach'] = np.exp(prob['dvach'])

In [20]:
prob.most_common()

[('news', 3.8562170896053503e-25), ('dvach', 8.260197121507218e-38)]

Работает получше. Мы воспользовались небольшим хаком - для слов или биграммов, которых не было у нас в словаре, прибавляли низкую вероятность. Исправить это по-нормальному - сложно, придется подробнее разбираться с вероятностями, сглаживаниями и заменой неизвестных слов. Если интрересно - в книге Журафского про это есть.

Теперь попробуем посчитать условную верояность с помощью триграммов. Тут мы добавляем еще один токен `<start>` в начало. Это нужно для того, чтобы могли строить предсказания начиная с первого слова. Ведь триграмы предсказывают вероятность на основе **двух** предыдущих слов, значит перед первым и вторым словом не хватает токенов.

In [21]:
phrase = 'Технические возможности устаревшего российского судна не позволили разгрузить его у терминала''Ныть надо меньше и работать больше.'
# phrase = 'как вы смотрите эту залупу, серьезно. в чем прикол ваще это смотреть?'
prob = Counter()
for ngram in ngrammer(['<start>', '<start>'] + normalize(phrase) + ['<end>'], n=3):
    word1, word2, word3 = ngram.split()
    
    if word1 in unigrams_dvach and ngram in trigrams_dvach:
        prob['dvach'] += np.log(trigrams_dvach[ngram]/unigrams_dvach[word1])
    else:
        prob['dvach'] += np.log(0.001)
    
    if word1 in unigrams_news and ngram in trigrams_news:
        prob['news'] += np.log(trigrams_news[ngram]/unigrams_news[word1])
    else:
        prob['news'] += np.log(0.001)

prob['news'] = np.exp(prob['news'])
prob['dvach'] = np.exp(prob['dvach'])

In [22]:
prob.most_common()

[('news', 5.5054854013773364e-45), ('dvach', 1.000000000000004e-51)]

Разница на 6 степеней получилась. А в биграмах была разница 13 степеней, то есть биграмы на этой задаче более разительную разницу показали почему-то.

Генерируем новый текст на основне биграмов:

In [18]:
matrix_dvach = np.zeros((len(unigrams_dvach), 
                   len(unigrams_dvach)))
id2word_dvach = list(unigrams_dvach)
word2id_dvach = {word:i for i, word in enumerate(id2word_dvach)}


for ngram in bigrams_dvach:
    word1, word2 = ngram.split()
    matrix_dvach[word2id_dvach[word1]][word2id_dvach[word2]] =  (bigrams_dvach[ngram]/
                                                                     unigrams_dvach[word1])



In [19]:
# создадим матрицу вероятностей перейти из 1 слов в другое
matrix_news = np.zeros((len(unigrams_news), 
                   len(unigrams_news)))

id2word_news = list(unigrams_news)
word2id_news = {word:i for i, word in enumerate(id2word_news)}


# вероятность расчитываем точно также
for ngram in bigrams_news:
    word1, word2 = ngram.split()
    matrix_news[word2id_news[word1]][word2id_news[word2]] =  (bigrams_news[ngram]/
                                                                     unigrams_news[word1])



Для генерации нам понадобится функция np.random.choice , которая выбирает случайный объект из заданных. Ещё в неё можно подать вероятность каждого объекта и она будет доставать по ним (не только максимальный по вероятности)

In [20]:

def generate(matrix, id2word, word2id, n=100, start='<start>'):
    text = []
    current_idx = word2id[start]
    
    for i in range(n):
        
        chosen = np.random.choice(matrix.shape[1], p=matrix[current_idx])
        text.append(id2word[chosen])
        
        if id2word[chosen] == '<end>':
            chosen = word2id['<start>']
        current_idx = chosen
    
    return ' '.join(text)

In [23]:
print(generate(matrix_dvach, id2word_dvach, word2id_dvach).replace('<end>', '\n'))

ответ 20 не может быть программистом и сделаю это говно как там ещё чтобы разговор которого хватает 
 позже всё ещё необычная мегумин 
 сёма ну она не гослинговский молчаливый кун с перла но больно анальные там юно на программу нихуя что-то с нуля пропуская кучу ненужной параши python php c кодер тян 
 о коля привет привет петя и хх 20 баксов миллионер ахаха в it сверхдержава блять 
 а как такового но они сложные только шахматы ходят на хуй говорю когда танцоры по 90 программистов это факт против которого я хз кто не останешься 
 наверное математик 



In [25]:
print(generate(matrix_news, id2word_news, word2id_news).replace('<end>', '\n'))

в подписных листах если угодно вопросы финансовой политике направленном правительством москвы напоминающий самодельное взрывное устройство 
 есть потери 
 пленка была объявлена большая колонна отступавшая по крайней мере расследования 
 а пока не прекращаютсявопреки четким требованиям верховных звонков в истории гражданской авиации аргентины 
 его мнению представителей американских ученых из центральных улиц бельгийской артиллерии руководивший обороной одного из россиян во фронт 
 кроме того кто сегодня что все неурядицы сейчас в очаге конфликта 250 долларов 
 как сообщает сегодня по сведениям в торговом комплексе охотный ряд на своей неотъемлемой частью при чем на интервенции до 3300 рублей 
 правда окраины


У меня получается генерировать тексты только на очень маленьком количестве предложений, до 1000, поэтому тексты дикий отстой. Если делать больше данных, то память ложится даже в колабе. Возможно я что-то не так делаю с колабом ....

In [27]:
matrix_dvach = np.zeros((len(unigrams_dvach), 
                   len(unigrams_dvach)))
id2word_dvach = list(unigrams_dvach)
word2id_dvach = {word:i for i, word in enumerate(id2word_dvach)}


for ngram in trigrams_dvach:
    word1, word2, word3 = ngram.split()
    matrix_dvach[word2id_dvach[word1]][word2id_dvach[word2]] =  (trigrams_dvach[ngram]/
                                                                     unigrams_dvach[word1])

In [32]:
# создадим матрицу вероятностей перейти из 1 слов в другое
matrix_news = np.zeros((len(unigrams_news), 
                   len(unigrams_news)))

id2word_news = list(unigrams_news)
word2id_news = {word:i for i, word in enumerate(id2word_news)}


# вероятность расчитываем точно также
for ngram in trigrams_news:
    word1, word2, word3 = ngram.split()
    matrix_news[word2id_news[word1]][word2id_news[word2]] =  (trigrams_news[ngram]/
                                                                     unigrams_news[word1])

In [29]:
def generate(matrix, id2word, word2id, n=100, start='<start>'):
    text = []
    current_idx = word2id[start]
    
    for i in range(n):
        
        chosen = np.random.choice(matrix.shape[1], p=matrix[current_idx])
        text.append(id2word[chosen])
        
        if id2word[chosen] == '<end>':
            chosen = word2id['<start>']
        current_idx = chosen
    
    return ' '.join(text)

In [33]:
print(generate(matrix_dvach, id2word_dvach, word2id_dvach).replace('<end>', '\n'))

ValueError: ignored

In [31]:
print(generate(matrix_news, id2word_news, word2id_news).replace('<end>', '\n'))

ValueError: ignored